In [9]:
import sys
from web3 import Web3
from misc import wrapper, load_yaml, load_json

#### LOAD CONFIG

In [10]:
config = load_yaml('./config.yml')

#### CONNECT

In [11]:
def connect():
    
    # CONNECT TO BLOCKCHAIN GATEWAY
    address = config.gateway_uri
    instance = Web3(Web3.WebsocketProvider(config.gateway_uri))
    
    # NOT RESPONDING
    if not instance.isConnected():
        raise Exception('COULD NOT CONNECT TO GATEWAY! ABORTING...')
    
    # OTHEREWISE, RETURN INSTANCE
    return instance

In [12]:
instance = connect()

In [14]:
instance.fromWei(3841357360894980500000001, 'ether')

Decimal('3841357.360894980500000001')

In [18]:
instance.eth.get_balance(config.accounts[0].public)

1000000000000000000000

In [27]:
instance.signMessage

AttributeError: 'Web3' object has no attribute 'signMessage'